<a href="https://colab.research.google.com/github/lucaspn017/elect_consumption/blob/main/Elect_Consump_Time__Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Energy Consumption Analysis. <h1>

- <h3> This program is intended to be a data calculation method for estimating the energy consumption for a group A unit. <h3>

- <h3> Basically, there are two times when the energy rate will change: peak time and off-peak. In this way, I need to separate the values for these two moments. <h3>

In [ ]:
import pandas as pd
import numpy as np
import datetime as datetime

In [ ]:
 # The data i received used '.' instead of ',' to separate decimal numbers.
 # It´s necessary to modify this.

 paths = glob('datas\*')

 for path in paths:
  with open(path,'r') as f:
    # The datas have names parcially equals, in that way, i
    # had to modify just one part.
    with open('original_datas' + path[12:-4] + 'csv', 'w') as f2:
      # I change the names of columns
      f2.write('Datetime,FaseA,FaseB,FaseC,Total\n')
      for i, line in enumerate(f.readlines()):

        if i != 0:
          row = line.replace(',','.').replace(';',',')
          f2.write(row)

In [ ]:
path_pa = glob('original_datas\*_first_*')[0]
path_pf = glob('original_datas\*_second_*')[0]

In [ ]:
path_pa = pd.read_csv(path_pa)

# Datetime is not the table index. I have to make it.
path_pa.index = pd.to_datetime(path_pa['Datetime'])
path_pa = path_pa.drop(['Datetime'], axis=1)
path_pa = path_pa.resample('10min').mean().fillna('pad')

path_pa['Time'] = pd.DataFrame(list(map(lambda x: path_pa.index.time(), path_pa.index)), index = path_pa)

In [ ]:
path_pf = pd.read_csv(path_pf)

path_pf.index = pd.to_datetime(path_pf['Datetime'])
path_pf = path_pa.drop(['Datetime'], axis= 1)
path_pf = path_pf.resample('10min').resample('1h').mean().fillna('pad')
path_pf['Time'] = pd.DataFrame(list(map(lambda x: path_pf.index.time(), path_pf.index)), index = path_pf)

In [ ]:
# There are different rates charged according to specific intervals in the day.

initial = datetime.strptime('18:30:00', '%H:%M:%S').time()
final = datetime.strptime('21:30:00', '%H:%M:%S').time()

In [ ]:
consump_op = pd.DataFrame(columns=['consump_low'])
consump_ip = pd.DataFrame(columns=['consump_high'])

# I receive datas that areh usually on the same year/month basis,
# but i thought it necessary to create a generic code.

for year in np.unique(np.array(list(map(lambda x: x.year, path_pa.index)))):
  for month in np.unique(np.array(list(map(lambda x: x.month, path_pa.index)))):
    for day in np.unique(np.array(list(map(lambda x: x.day, path_pa.index)))):
      for hour in np.unique(list(map(lambda x: x.hour, path_pa.index))):
        # These values must be operated with corresponding periods.

        func_ip = lambda x: (x['Time']>=initial) & (x['Time']<=final) & (x.index.year==year) & (x.index.month==month) & (x.index.day==day) & (x.index.hour==hour)
        func_op = lambda x: ((x['Time']<initial) | (x['Time']>final)) & (x.index.year==year) & (x.index.month==month) & (x.index.day==day) & (x.index.hour==hour)

        if(hour>=18)&(hour<=21):
          consump_ip.loc[pd.to_datetime(datetime(year=year,month=month,day=day,hour=hour,minute=0))]=np.trapz(path_pf.loc[func_ip]['Total'].values, dx=1/6)
        if(hour<18)|(hour>21):
          consump_op.loc[pd.to_datetime(datetime(year=year,month=month,day=day,hour=hour,minute=0))]=np.trapz(path_pa.loc[func_op]['Total'].values, dx=1/6)


In [ ]:
consump_ip['Time'] = pd.DataFrame(list(map(lambda x: x.time(), consump_ip.index)), index=consump_ip.index)
consump_op['Time'] = pd.DataFrame(list(map(lambda x: x.time(), consump_op.index)), index=consump_op.index)

In [ ]:
consump_op['PF'] = path_pf.loc[list(consump_op.index)]['Total']
consump_ip['PF'] = path_pf.loc[list(consump_ip.index)]['Total']

In [ ]:
t1 = datetime.strptime('23:30:00', '%H:%M:%S').time()
t2 = datetime.strptime('06:30:00', '%H:%M:%S').time()

In [ ]:
# There´s a norma proposed by ANEEL called: Normative Resolution n° 1000.
# It contains the calculus of over reactive, used in expression below.

over_ip = consump_ip.apply(lambda x: (x['consump_high']*(0.92/abs(x['PF'])-1)) if ((x['PF']<0.92) & (x['PF']>0) & ((x['Time']<t1) & (x['Time']>t2))) | ((x['PF']>-0.92) & (x['PF']<0) & ((x['Time']>t1) | (x['Time']<t2))) else 0 , axis=1)
over_op = consump_op.apply(lambda x: (x['consump_low']*(0.92/abs(x['PF'])-1)) if ((x['PF']<0.92) & (x['PF']>0) & ((x['Time']<t1) & (x['Time']>t2))) | ((x['PF']>-0.92) & (x['PF']<0) & ((x['Time']>t1) | (x['Time']<t2))) else 0 , axis=1)

In [ ]:
over_ip.sum()
over_op.sum()

In [ ]:
consump_ip['consump_low'].sum()